<a href="https://colab.research.google.com/github/PritamGoyal/Image-steganography/blob/main/image_steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install Crypto
!pip install colorama==0.4.4
!pip install commonmark==0.9.1
!pip install Pillow==8.1.0
!pip install pycryptodome==3.9.9
!pip install pyfiglet==0.8.post1
!pip install Pygments==2.7.4
!pip install rich==9.10.0
!pip install termcolor==1.1.0
!pip install typing-extensions==3.7.4.3

from PIL import Image
import os.path
from os import path
import math
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
from Crypto import Random
import base64
from PIL import ImageOps
import imghdr
from rich import print
from rich.console import Console
from rich.table import Table
from rich.progress import track
import getpass
import sys
from math import ceil

DEBUG = False
console = Console()
headerText = "M6nMjy5THr2J"
SUPPORTED_FORMATS = ['PNG', 'JPEG', 'JPG', 'TIFF', 'BMP', 'JFIF', 'WEBP']

def encrypt(key, source, encode=True):
    key = SHA256.new(key).digest()
    IV = Random.new().read(AES.block_size)
    encryptor = AES.new(key, AES.MODE_CBC, IV)
    padding = AES.block_size - len(source) % AES.block_size
    source += bytes([padding]) * padding
    data = IV + encryptor.encrypt(source)
    return base64.b64encode(data).decode() if encode else data

def decrypt(key, source, decode=True):
    if decode:
        source = base64.b64decode(source.encode())
    key = SHA256.new(key).digest()
    IV = source[:AES.block_size]
    decryptor = AES.new(key, AES.MODE_CBC, IV)
    data = decryptor.decrypt(source[AES.block_size:])
    padding = data[-1]
    if data[-padding:] != bytes([padding]) * padding:
        raise ValueError("Invalid padding...")
    return data[:-padding]


def convertToRGB(img):
    try:
        if img.mode == 'RGB':
            return img
        rgba_image = img.convert('RGBA')
        background = Image.new("RGB", rgba_image.size, (255, 255, 255))
        background.paste(rgba_image)
        print("[yellow]Converted image to RGB [/yellow]")
        return background
    except Exception as e:
        print("[red]Couldn't convert image to RGB [/red]- %s" % e)
        return None


def getPixelCount(img):
    width, height = Image.open(img).size
    return width*height

def calculate_required_dimensions(message_length, max_pixels_per_character):

    pixels_needed = message_length * max_pixels_per_character
    width = int((pixels_needed ** 0.5) + 0.5)
    height = (pixels_needed + width - 1) // width
    return width, height

def compressImage(image_path):

    try:
      try:
        img = Image.open(image_path)
      except Exception as e:
        print(f"Error opening the image: {e}")

      if img.format.upper() not in SUPPORTED_FORMATS:
          raise ValueError(f"Unsupported image format: {image.format}")

      filename, extension = os.path.splitext(image_path)
      compressed_path = f"{filename}-compress.png"

      if img.format == "PNG":
        print(f"[yellow]Image is already in PNG format. Using it as is. [/yellow]")
        return compressed_path
      else:
        img.save(compressed_path, "PNG")
        print(f"[yellow]Image compressed and saved to {compressed_path}[/yellow]")
        return compressed_path
    except Exception as e:
      print(f"[red]Error compressing image: {e}[/red]")
      return None

def encodeImage(image,message,filename):
    with console.status("[green]Encoding image..") as status:
        try:
            width, height = image.size
            pix = image.getdata()

            current_pixel = 0
            tmp=0

            x=0
            y=0
            for ch in message:
                binary_value = format(ord(ch), '08b')


                p1 = pix[current_pixel]
                p2 = pix[current_pixel+1]
                p3 = pix[current_pixel+2]

                three_pixels = [val for val in p1+p2+p3]

                for i in range(0,8):
                    current_bit = binary_value[i]

                    if current_bit == '0':
                        if three_pixels[i]%2!=0:
                            three_pixels[i]= three_pixels[i]-1 if three_pixels[i]==255 else three_pixels[i]+1
                    elif current_bit == '1':
                        if three_pixels[i]%2==0:
                            three_pixels[i]= three_pixels[i]-1 if three_pixels[i]==255 else three_pixels[i]+1

                current_pixel+=3
                tmp+=1

                if(tmp==len(message)):

                    if three_pixels[-1]%2==0:
                        three_pixels[-1]= three_pixels[-1]-1 if three_pixels[-1]==255 else three_pixels[-1]+1
                else:
                    if three_pixels[-1]%2!=0:
                        three_pixels[-1]= three_pixels[-1]-1 if three_pixels[-1]==255 else three_pixels[-1]+1


                if DEBUG:
                    print("Character: ",ch)
                    print("Binary: ",binary_value)
                    print("Three pixels before mod: ",three_pixels)
                    print("Three pixels after mod: ",three_pixels)


                three_pixels = tuple(three_pixels)

                st=0
                end=3

                for i in range(0,3):
                    if DEBUG:
                        print("Putting pixel at ",(x,y)," to ",three_pixels[st:end])

                    image.putpixel((x,y), three_pixels[st:end])
                    st+=3
                    end+=3

                    if (x == width - 1):
                        x = 0
                        y += 1
                    else:
                        x += 1

            encoded_filename = filename.split('.')[0] + "-enc.png"
            image.save(encoded_filename)
            print("\n")
            print("[yellow]Original File: [u]%s[/u][/yellow]"%filename)
            print("[green]Image encoded and saved as [u][bold]%s[/green][/u][/bold]"%encoded_filename)

        except Exception as e:
            print("[red]An error occured - [/red]%s"%e)
            sys.exit(0)

def decodeImage(image):
    with console.status("[green]Decoding image..") as status:
        try:
            pix = image.getdata()
            current_pixel = 0
            decoded=""
            data_found = False
            while True:
                try:
                  binary_value=""
                  p1 = pix[current_pixel]
                  p2 = pix[current_pixel+1]
                  p3 = pix[current_pixel+2]
                  three_pixels = [val for val in p1+p2+p3]

                  for i in range(0,8):
                      if three_pixels[i]%2==0:
                          binary_value+="0"
                      elif three_pixels[i]%2!=0:
                          binary_value+="1"


                  binary_value.strip()
                  ascii_value = int(binary_value,2)
                  decoded+=chr(ascii_value)
                  current_pixel+=3

                  if DEBUG:
                      print("Binary: ",binary_value)
                      print("Ascii: ",ascii_value)
                      print("Character: ",chr(ascii_value))

                  if three_pixels[-1]%2!=0:
                      data_found =True
                      break
                except(ValueError, TypeError) as e:
                    raise Exception(f"[red]Error processing image data. The provided image might not be a valid stego image.[/red]")
                    break

            if not data_found:
                raise ValueError("No data found in the provided image.")


            return decoded
        except Exception as e:
            print("[red]An error occured - [/red]%s"%e)
            sys.exit()

def main():


    while True:
        print("[cyan]Choose one: [/cyan]")
        op = int(input("1. Encode\n2. Decode\n3. Exit\n>>"))

        if op == 1:

            print("[cyan]Message to be hidden: [/cyan]")
            message = input(">>")
            if not message:
                raise ValueError("Message cannot be empty.")

            message = headerText + message

            max_pixels_per_character = 3 * 8
            required_width, required_height = calculate_required_dimensions(len(message), max_pixels_per_character)
            print(f"[yellow]Minimum required dimensions for cover image: [bold]{required_width} x {required_height} pixels[/bold][/yellow]")

            print("[cyan]Image path (with extension): [/cyan]")
            img = input(">>")


            if not (path.exists(img)):
                raise Exception("Image not found!")

            image_format = imghdr.what(img)
            if image_format not in {'png', 'jpg', 'jpeg', 'tiff', 'bmp', 'jfif', 'webp'}:
                raise ValueError("Invalid image format. Supported formats are: png, jpg, jpeg, tiff, bmp, jfif, webp.")

            img_width, img_height = Image.open(img).size
            compressed_image_path = compressImage(img)

            if img_width < required_width or img_height < required_height:
               raise Exception(f"The provided image dimensions ({img_width} x {img_height} pixels) are smaller than the suggested dimensions ({required_width} x {required_height} pixels). Please provide an image with suitable dimensions.")



            max_message_length = (img_width * img_height * 3) // 8 - len(headerText)
            if len(message) > max_message_length:
                raise Exception(f"Given message is too long to be encoded in the image.\nTry resizing the image to accommodate a larger message.\nCurrent maximum message length: {max_message_length} characters.")



            password = None
            while 1:
                print("[cyan]Password to encrypt: [/cyan]")
                password = getpass.getpass(">>")

                if password:
                  print("[cyan]Re-enter Password: [/cyan]")
                  confirm_password = getpass.getpass(">>")
                  if password != confirm_password:
                      print("[red]Passwords don't match try again [/red]")
                  else:
                      break
                else:
                  raise ValueError("Password cannot be empty.")

            cipher = ""
            if password != "":
                cipher = encrypt(key=password.encode(), source=message.encode())

                cipher = headerText + cipher
            else:
                cipher = message

            if DEBUG:
                print("[yellow]Encrypted : [/yellow]", cipher)

            try:
                image = Image.open(img)
            except Exception as e:
                print(f"Error opening the image: {e}")

            image.filename=img
            print("[yellow]Image Mode: [/yellow]%s"%image.mode)
            if image.mode!='RGB':
                image = convertToRGB(image)
            newimg = image.copy()
            encodeImage(image=newimg,message=cipher,filename=img)

        elif op == 2:
            print("[red]Stego Image must be in png format[/red]")
            print("[cyan]Image path (with extension): [/cyan]")
            img = input(">>")
            if not (path.exists(img)):
                raise Exception("Image not found!")

            password=None
            while True:
              print("[cyan]Enter password : [/cyan]")
              password_input = getpass.getpass(">>")
              if password_input:
                password = password_input
                break
              else:
                raise ValueError("Password cannot be empty.")

            try:
                image = Image.open(img)
            except Exception as e:
                print(f"Error opening the image: {e}")


            if image.format.upper() != 'PNG':
                raise ValueError("The stego image must be in PNG format.")



            cipher = decodeImage(image)

            header = cipher[:len(headerText)]

            if header.strip() != headerText:
                print("[red]Invalid data![/red]")
                sys.exit(0)

            print()

            if DEBUG:
                print("[yellow]Decoded text: %s[/yellow]" % cipher)

            decrypted = ""

            if password != "":
                cipher = cipher[len(headerText):]
                print("cipher : ", cipher)
                try:
                    decrypted = decrypt(key=password.encode(), source=cipher)
                except Exception as e:
                    print("[red]Wrong password![/red]")
                    sys.exit(0)

            else:
                decrypted = cipher

            header = decrypted.decode()[:len(headerText)]

            if header != headerText:
                print("[red]Wrong password![/red]")
                sys.exit(0)

            decrypted = decrypted[len(headerText):].decode("utf-8")

            print("[green]Decoded Text: \n[bold]%s[/bold][/green]" % decrypted)

        elif op == 3:
            break

        else:
            print("Incorrect Choice")
        print("\n")


if __name__ == "__main__":
    os.system('cls' if os.name == 'nt' else 'clear')

    print()
    print()

    main()



Choose one: 
1. Encode
2. Decode
3. Exit
>>1
Message to be hidden: 
>>meoww meowkhvvm kbkdgcwkgkg kjbgkghb
Minimum required dimensions for cover image: 34 x 34 pixels
Image path (with extension): 
>>/content/LINKEDIN BACKGROUNGIMAGE.png
Image is already in PNG format. Using it as is. 
Password to encrypt: 
>>··········
Re-enter Password: 
>>··········
Image Mode: RGBA
Converted image to RGB 


Original File: /content/LINKEDIN BACKGROUNGIMAGE.png
Image encoded and saved as /content/LINKEDIN BACKGROUNGIMAGE-enc.png


Choose one: 
1. Encode
2. Decode
3. Exit
>>2
Stego Image must be in png format
Image path (with extension): 
>>/content/LINKEDIN BACKGROUNGIMAGE-enc.png
Enter password : 
>>··········

cipher :  6ljhxoP1SK69PPgbl8yHpOjw74/jgXFkCaHcxcPXQ8HsaAovccM6iHslG0ZvYSzWuVwfsR
jw5ZUMG1fjhI0YkFSJU0511wBWtO/asQ6oCGk=
Decoded Text: 
meoww meowkhvvm kbkdgcwkgkg kjbgkghb


Choose one: 
1. Encode
2. Decode
3. Exit
>>3
